In [4]:
from static_checks.Checker import *
from static_checks.MiniInstantiator import *
from common.datatypes import ForecastingQuestion
from forecasters.consistent_forecaster import ConsistentForecaster
from uuid import UUID
import uuid
import datetime

P = ForecastingQuestion(
        id=UUID("a874f3b3-85b0-4383-a009-efda1ff26357"),
        title="Will a member of the Trump family be the Republican nominee for the US Presidency in 2024?",
        body="Resolution Criteria\nResolves Yes if one of these individuals is the Republican nominee for the US Presidency in 2024:\nDonald Trump\nDonald Trump Jr.\nEric Trump\nIvanka Trump\nTiffany Trump\nMelania Trump\nBarron Trump\n",
        resolution_date=datetime.datetime(2024, 12, 31, 0, 0),
        question_type="binary",
        data_source="synthetic_inst",
        created_date=None,
        url=None,
        metadata=None,
        resolution=None,
    )

Q = ForecastingQuestion(
        id=UUID("a874f3b3-85b0-4383-a009-efda1ff26357"),
        title="Will Trump invent AGI in 2024?",
        body="Resolution Criteria\nResolves Yes if Trump is responsible for the creation of Artificial General Intelligence in 2024. Resolves No otherwise.",
        resolution_date=datetime.datetime(2024, 12, 31, 0, 0),
        question_type="binary",
        data_source="synthetic_inst",
        created_date=None,
        url=None,
        metadata=None,
        resolution=None,
    )

R = ForecastingQuestion(
    id=uuid.uuid4(),
    title="Will the Democrats win the 2028 presidential election in the U.S.?",
    body="Resolves Yes if the Democratic candidate wins the 2028 presidential election in the United States, and No otherwise. "
    "If the candidate is not a member of the Democratic party, but the party endorses them, resolves Yes. "
    "If the party ceases to exist and a candidate from another party wins, resolves No. "
    "If the election is not held between October and December 2028, or if there ceases to be an office of "
    "President of the United States, resolves N/A.",
    question_type="binary",
    resolution_date=datetime.datetime(2028, 12, 31),
    data_source="synthetic",
    url=None,
    metadata={"topics": ["Politics", "Elections", "United States"]},
    resolution=None,
)

bq_tuple = {
    "P": P,
    "Q": Q,
}


x= await CondChecker().instantiate(bq_tuple,model="gpt-4o-mini")


print(x)

Base sentences: {'P': ForecastingQuestion(id=UUID('a874f3b3-85b0-4383-a009-efda1ff26357'), title='Will a member of the Trump family be the Republican nominee for the US Presidency in 2024?', body='Resolution Criteria\nResolves Yes if one of these individuals is the Republican nominee for the US Presidency in 2024:\nDonald Trump\nDonald Trump Jr.\nEric Trump\nIvanka Trump\nTiffany Trump\nMelania Trump\nBarron Trump\n', resolution_date=datetime.datetime(2024, 12, 31, 0, 0), question_type='binary', data_source='synthetic_inst', created_date=None, url=None, metadata=None, resolution=None)}
Instantiating with instantiator Trivial
Instantiation attempt 0
Verifying output: P=ForecastingQuestion(id=UUID('4efc6a12-60b8-4f03-a885-6d01aa71b3ec'), title='Will a member of the Trump family be the Republican nominee for the US Presidency in 2024?', body='Resolution Criteria\nResolves Yes if one of these individuals is the Republican nominee for the US Presidency in 2024:\nDonald Trump\nDonald Trump J

In [1]:
# %%

from static_checks.Checker import *
from time import time

checkers = {
    "para": {"answers": {"P": 0.4, "para_P": 0.8}, "checker": ParaphraseChecker()},
    "neg": {"answers": {"P": 0.6, "not_P": 0.8}, "checker": NegChecker()},
    "cond": {
        "answers": {"P": 0.15, "Q_given_P": 0.6, "P_and_Q": 0.15},
        "checker": CondChecker(),
    },
    "andor": {
        "answers": {"P": 0.6, "Q": 0.6, "P_and_Q": 0.55, "P_or_Q": 0.6},
        "checker": AndOrChecker(),
    },
    "and": {
        "answers": {"P": 0.15, "Q": 0.1, "P_and_Q": 0.1},
        "checker": AndChecker(),
    },
    "or": {
        "answers": {"P": 0.2, "Q": 0.3, "P_or_Q": 0.3},
        "checker": OrChecker(),
    },
    "but": {
        "answers": {"P": 0.6, "Q_and_not_P": 0.3, "P_or_Q": 0.94},
        "checker": ButChecker(),
    },
    "condcond": {
        "answers": {
            "P": 0.5,
            "Q_given_P": 0.5,
            "R_given_P_and_Q": 0.5,
            "P_and_Q_and_R": 0.16,
        },
        "checker": CondCondChecker(),
    },
    "cons": {"answers": {"P": 0.5, "cons_P": 0.45}, "checker": ConsequenceChecker()},
    "ee": {
        "answers": {
            "P": 0.5,
            "P_given_Q": 0.25,
            "P_given_not_Q": 0.4,
            "Q": 0.15,
        },
        "checker": ExpectedEvidenceChecker(),
    },
}

for k, v in checkers.items():
    checker = v["checker"]
    # if k not in ["and", "or"]:
    #     continue
    answers = v["answers"]
    print("Checker:", k)
    time0 = time()
    result_plain = checker.max_min_arbitrage(answers)
    time1 = time()
    print("No kwargs:", result_plain, "\nTime:", time1 - time0)
    # result_weighted = checker.max_min_arbitrage(answers, scoring=[0.5, 0.2])
    # print("Weighted:", result_weighted, "\nTime:", time() - time1)
    try:
        result_de = checker.de_method(answers, tmax=10, euler=False)
    except Exception as e:
        result_de = None
    time2 = time()
    print("DE:", result_de, "\nTime:", time2 - time1)
    try:
        result_de = checker.de_method(answers, tmax=1, euler=False)
    except Exception as e:
        result_de = None
    time3 = time()
    print("DE:", result_de, "\nTime:", time3 - time2)
    print("\n")



/home/abhim/consistency-forecasting/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


LOCAL_CACHE: None
max_concurrent_queries set for global semaphore: 20
Generic logs directory already exists at /home/abhim/consistency-forecasting/logs
Resetting global semaphore, max concurrent queries: 20

 NO_CACHE 


 NO_CACHE 


 NO_CACHE 


 NO_CACHE 



/home/abhim/consistency-forecasting/venv/lib/python3.11/site-packages/pydantic/_internal/_model_construction.py:66: UserWarning: `check_question_type` overrides an existing Pydantic `@field_validator` decorator
  warnings.warn(f'`{k}` overrides an existing Pydantic `{existing.decorator_info.decorator_repr}` decorator')
/home/abhim/consistency-forecasting/venv/lib/python3.11/site-packages/pydantic/_internal/_model_construction.py:66: UserWarning: `check_question_type` overrides an existing Pydantic `@field_validator` decorator
  warnings.warn(f'`{k}` overrides an existing Pydantic `{existing.decorator_info.decorator_repr}` decorator')


Checker: and


LinAlgError: Last 2 dimensions of the array must be square

In [ ]:
# %%

from forecasters.consistent_forecaster import ConsistentForecaster
from forecasters.basic_forecaster import BasicForecaster
from common.datatypes import ForecastingQuestion
from static_checks.Checker import *
from time import time

cf = ConsistentForecaster(
    BasicForecaster(),
    checks=[
        NegChecker(),
        AndOrChecker(),
        CondChecker(),
        ParaphraseChecker(),
    ],
)
fq = ForecastingQuestion(
    title="Will Manhattan have a skyscraper a mile tall by 2030?",
    body=(
        "Resolves YES if at any point before 2030, there is at least "
        "one building in the NYC Borough of Manhattan (based on current "
        "geographic boundaries) that is at least a mile tall."
    ),
    resolution_date="2030-01-01T00:00:00",
    question_type="binary",
    data_source="manifold",
    url="https://www.metaculus.com/questions/12345/",
    metadata={"foo": "bar"},
    resolution=None,
)

import asyncio

# x = cf.call(fq)
x = asyncio.run(cf.call_async(
    fq,
    bq_func_kwargs={"model": "gpt-4o-mini"},
    instantiation_kwargs={"model": "gpt-4o-mini"},
    model="gpt-4o-mini",
))
# x = asyncio.run(cf.call_async(fq))

In [ ]:
# %%

print(f"{x=}")

raise

In [ ]:
# %%

from static_checks.Checker import *
from time import time

checkers = {
    "para": {"answers": {"P": 0.4, "para_P": 0.43}, "checker": ParaphraseChecker()},
    "neg": {"answers": {"P": 0.6, "not_P": 0.57}, "checker": NegChecker()},
    "cond": {
        "answers": {"P": 0.15, "Q_given_P": 0.6, "P_and_Q": 0.15},
        "checker": CondChecker(),
    },
    "andor": {
        "answers": {"P": 0.6, "Q": 0.6, "P_and_Q": 0.55, "P_or_Q": 0.6},
        "checker": AndOrChecker(),
    },
    "but": {
        "answers": {"P": 0.6, "Q_and_not_P": 0.3, "P_or_Q": 0.94},
        "checker": ButChecker(),
    },
    "condcond": {
        "answers": {
            "P": 0.5,
            "Q_given_P": 0.5,
            "R_given_P_and_Q": 0.5,
            "P_and_Q_and_R": 0.16,
        },
        "checker": CondCondChecker(),
    },
    "cons": {"answers": {"P": 0.5, "cons_P": 0.45}, "checker": ConsequenceChecker()},
}

for k, v in checkers.items():
    checker = v["checker"]
    answers = v["answers"]
    if k != "cons":
        continue
    print("Checker:", k)
    time0 = time()
    result_shgo = checker.max_min_arbitrage(answers, methods=("shgo",))
    time1 = time()
    print("SHGO:", result_shgo, "\nTime:", time1 - time0)
    result_diff_evol = checker.max_min_arbitrage(
        answers, methods=("differential_evolution",)
    )
    time2 = time()
    print("Diff Evolution:", result_diff_evol, "\nTime:", time2 - time1)
    result_dual_annealing = checker.max_min_arbitrage(
        answers, methods=("dual_annealing",)
    )
    time3 = time()
    print("Dual Annealing:", result_dual_annealing, "\nTime:", time3 - time2)
    result_basinhopping = checker.max_min_arbitrage(answers, methods=("basinhopping",))
    time4 = time()
    print("Basinhopping:", result_basinhopping, "\nTime:", time4 - time3)
    result_plain = checker.max_min_arbitrage(answers)
    time5 = time()
    print("No kwargs:", result_plain, "\nTime:", time5 - time4)
    print("\n\n")


# print('----------------')
# print(CondChecker().min_arbitrage(ex, exa))
# print('----------------')
# print(CondChecker().arbitrage(outcome1, ex, exa))
# print(CondChecker().arbitrage(outcome2, ex, exa))
# print(CondChecker().arbitrage(outcome3, ex, exa))
# print('----------------')

In [ ]:
# %%

import json
from common.datatypes import (
    ForecastingQuestion_stripped,
    ForecastingQuestion,
    Prob_cot,
    Prob,
    PlainText,
)
from common.llm_utils import query_api_chat_sync, query_api_chat_sync_native
import os

fq = ForecastingQuestion(
    title="Will Manhattan have a skyscraper a mile tall by 2030?",
    body=(
        "Resolves YES if at any point before 2030, there is at least "
        "one building in the NYC Borough of Manhattan (based on current "
        "geographic boundaries) that is at least a mile tall."
    ),
    resolution_date="2030-01-01T00:00:00",
    question_type="binary",
    data_source="manifold",
    url="https://www.metaculus.com/questions/12345/",
    metadata={"foo": "bar"},
    resolution=None,
)

fqs = ForecastingQuestion_stripped(
    title="Will Manhattan have a skyscraper a mile tall by 2030?",
    body=(
        "Resolves YES if at any point before 2030, there is at least "
        "one building in the NYC Borough of Manhattan (based on current "
        "geographic boundaries) that is at least a mile tall."
    ),
)

print(fqs.__str__())

In [ ]:
# %%

# os.environ["USE_OPENROUTER"] = "True"
messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant. Summarize the question for the user.",
    },
    {"role": "user", "content": fq.__str__()},
]
# response = query_api_chat_sync(messages=messages, verbose=True, model="mistralai/mistral-large")